<a href="https://colab.research.google.com/github/Bellafqira/DICTION/blob/master/TP_Paillier_correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalation de conda et Gmpy2
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge gmpy2

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:24
🔁 Restarting kernel...
Solving environment: | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - gmpy2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    colorama-0.4.6             |     pyhd8ed1ab_0          25 KB  conda-forge
    conda-22.11.1              |   py38h578d9bd_1         905 KB  conda-forge
    gmp-6.2.1                  |       h58526e2_0         806 KB  conda-forge
    gmpy2-2.1.2                |   py38h793c122_1         223 KB  conda

In [ ]:
# import Packages
from gmpy2 import root, mpfr, cbrt, sqrt, c_div, is_prime, random_state, mpz_urandomb, bit_set, next_prime, bit_length, num_digits, mpz_random, gcd, invert, powmod, mpz, is_prime
import time
import random

### Génération de nombres premiers

In [ ]:
def get_prime(size): # size la taille en binaire
    seed = random_state(time.time_ns())
    p = mpz_urandomb(seed ,size)
    p = p.bit_set(size - 1)
    return next_prime(p)

### Génération de clé publique et privée de El Gamal

In [ ]:
def get_paillier_keys(size):
    p = get_prime(size//2)
    while True:
        q = get_prime(size//2)
        N =  p*q
        phi = (p-1)*(q-1)
        if gcd(N, phi)== 1 and p!=q:
            break
    g = 1 + N
    pub_key = N, g
    priv_key = phi, max(p,q), min(p,q)
    return pub_key, priv_key

In [ ]:
# Test
pub_key, priv_key  = get_paillier_keys(512)
print("N==", pub_key[0])
print("g==", pub_key[1])
print("phi==", priv_key[0])
print("p==", priv_key[1])
print("q==", priv_key[2])

N== 7109957788711517455864394470904371183022728013619609915324934706989880549927393573428801830391839492224387171671888082463122660662632531795323774019909059
g== 7109957788711517455864394470904371183022728013619609915324934706989880549927393573428801830391839492224387171671888082463122660662632531795323774019909060
phi== 7109957788711517455864394470904371183022728013619609915324934706989880549927224924894557351016630578202452967419876674226582293389281478486682042985890280
p== 85114353820846663386895193943636157088210693380122854363292621015092627489711
q== 83534180423632711822018827990568094923197543160244418987760687626638406529069


### Fonctions de chiffrement et de déchiffrement

In [ ]:
def get_r(pub_key):
    while True:
        seed = random_state(time.time_ns())
        r = mpz_random(seed, pub_key)
        if gcd(r, pub_key)== 1:
            break
    return r

def paillier_encrypt(message, pub_key):
    # get random value r
    r = get_r(pub_key[0])
    # encrypt the message
    N2 = pub_key[0] ** 2
    r = powmod(r, pub_key[0], N2)
    c = powmod(pub_key[1], message, N2)
    c = c*r % N2
    return c

def paillier_decrypt(enc, priv_key, pub_key):
    N2 = pub_key ** 2
    phiInv = invert(priv_key, pub_key)
    m = powmod(enc, priv_key, N2)
    m = m - 1
    m = m//pub_key
    m = m * phiInv % pub_key
    return m

def paillier_decrypt_CRT(enc, priv_key, pub_key):
    xp = powmod(enc, priv_key[0], priv_key[1]**2)
    xq = powmod(enc, priv_key[0], priv_key[2]**2)
    Invq = invert(priv_key[2]**2, priv_key[1]**2)
    x = ((Invq*(xp-xq))% priv_key[1]**2)*priv_key[2]**2 +  xq
    m = ((x-1)//pub_key[0]*invert(priv_key[0], pub_key[0])) % pub_key[0]
    return m

In [ ]:
# Test
message = mpz(102132100)
print("clear message = ", message)
enc = paillier_encrypt(message, pub_key)
print("encrypted message = ", enc)
m = paillier_decrypt(enc, priv_key[0], pub_key[0])
print("decrypted message = ", m)
m_crt = paillier_decrypt_CRT(enc, priv_key, pub_key)
print("decrypted message by CRT = ", m)

clear message =  102132100
encrypted message =  20127269663940869913124271485218848363216261539492908601281218041861669382904284304002525532870805473590450741650323216434701596147068027139203458375489592868610337178515491374247640394298676735594126070944537111486805694062182593178723795702296386632140359769326200945694927544842955991631217184741611767071
decrypted message =  102132100
decrypted message by CRT =  102132100
